In [1]:
import numpy as np

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [3]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [4]:
# def calc_logloss_modify(y, y_pred):
#     err = np.zeros(y_pred.shape)

#     err[y_pred==0] = np.inf + (1.0 - y[y_pred==0]) * np.log(1.0 - y_pred[y_pred==0])
#     err[y_pred==1] = y[y_pred==1] * np.log(y_pred[y_pred==1]) + np.inf
#     err[(y_pred!=0) & (y_pred!=1)] = y[(y_pred!=0) & (y_pred!=1)] * np.log(y_pred[(y_pred!=0) & (y_pred!=1)]) + \
#                                      (1.0 - y[(y_pred!=0) & (y_pred!=1)]) * np.log(1.0 - y_pred[(y_pred!=0) & (y_pred!=1)])
#     err = - np.mean(err)
#     return err


def calc_logloss_modify(y, y_pred):
    const = 1e-8
    err = - np.mean(y * np.log(y_pred + const) + (1.0 - y) * np.log(1.0 - y_pred + const))
    return err

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [5]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [6]:
def eval_model(X, y, iterations, alpha=1e-4, log=True):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss_modify(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0 and log:
            print(i, W, err)
    return W

In [7]:
W = eval_model(X, y, 2000, alpha=1e-6)
W

200 [ 0.49661415 -0.1385243   0.55568854  1.52288986] 9.210340366976185
400 [ 0.49651415 -0.1387843   0.46368854  1.52274986] 9.210340366976185
600 [ 0.49641415 -0.1390443   0.37168854  1.52260986] 9.210340366976185
800 [ 0.49631415 -0.1393043   0.27968854  1.52246986] 9.210340366976185
1000 [ 0.49621415 -0.1395643   0.18768854  1.52232986] 9.210340366976185
1200 [ 0.49611415 -0.1398243   0.09568854  1.52218986] 9.210340366976185
1400 [ 0.49601419 -0.14008427  0.00370585  1.52204989] 3.0526898857062825
1600 [ 0.49599251 -0.14009064 -0.00243037  1.52204931] 0.6047441637467333
1800 [ 0.49597611 -0.1400782  -0.00243041  1.52205868] 0.6047416048649721
2000 [ 0.4959597  -0.14006576 -0.00243045  1.52206805] 0.6047390462611573


array([ 0.4959597 , -0.14006576, -0.00243045,  1.52206805])

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [8]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(W @ X)
    return y_pred_proba

In [9]:
calc_pred_proba(W, X)

array([0.65987503, 0.54404739, 0.8080094 , 0.60340964, 0.40029095,
       0.51840875, 0.67176111, 0.23158333, 0.68659747, 0.69625237])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [10]:
def calc_pred(W, X, threshold=0.5):

    m = X.shape[1]
    
    y_pred = np.zeros((1, m))
    W = W.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(W.T, X))

    for i in range(A.shape[1]):
        if (A[:,i] > threshold): 
            y_pred[:, i] = 1
        elif (A[:,i] <= threshold):
            y_pred[:, i] = 0
    
    return y_pred

In [11]:
calc_pred(W, X)

array([[1., 1., 1., 1., 0., 1., 1., 0., 1., 1.]])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [12]:
def get_all_score(W, X):
    
    y_pred = calc_pred(W, X)[0]
    accuracy = 100.0 - np.mean(np.abs(y_pred - y)*100.0)
    
    conf_matrix = np.array([[0, 0], [0, 0]])
    
    a = y_pred.astype(np.int32)
    b = y.astype(np.int32)
    
    conf_matrix[0][0] = np.sum(a & b)    # TP 
    conf_matrix[0][1] = np.sum(a & ~b)    # FP
    conf_matrix[1][0] = np.sum(~a & b)    # FN
    conf_matrix[1][1] = np.sum(~a & ~b + 2)    # TN
    
    precision = conf_matrix[0][0]/(conf_matrix[0][0] + conf_matrix[0][1])
    recall = conf_matrix[0][0]/(conf_matrix[0][0] + conf_matrix[1][0])
    f1 = (2 * precision * recall)/(precision + recall)

    return accuracy, conf_matrix, precision, recall, f1

In [13]:
score = get_all_score(W, X)
print(f"""accuracy = {score[0]:>5}
confusion matrix:\n{score[1]}
precision = {score[2]:>4}
recall = {score[3]:>7}
f1 = {round(score[4], 5):>15}""")

accuracy =  50.0
confusion matrix:
[[4 4]
 [1 1]]
precision =  0.5
recall =     0.8
f1 =         0.61538


6. Могла ли модель переобучиться? Почему?

Да, наблюдений очень мало и есть выбросы в данных.

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [14]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-4, log=True):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss_modify(y, y_pred)
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + lambda_/2 * np.sign(W))
        if i % (iterations / 10) == 0 and log:
            print(i, W, err)
    return W

In [15]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-4, log=True):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss_modify(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0 and log:
            print(i, W, err)
    return W

In [16]:
eval_model_l1(X, y, 4000)

400 [ 0.49342182 -0.14540536 -0.03178787  1.5297142 ] 9.210340238190422
800 [ 0.49088643 -0.15082465 -0.02915924  1.53760621] 9.21034033144906
1200 [ 0.48822854 -0.1560587  -0.03613315  1.54539872] 9.210339232334823
1600 [ 0.48551236 -0.16131615 -0.07224638  1.5531292 ] 0.9733282304800962
2000 [ 0.4828483  -0.16652955 -0.07180663  1.5609657 ] 0.9643234375637333
2400 [ 0.48031145 -0.17121402  0.09070143  1.56905563] 2.7201470868948925
2800 [ 0.47753349 -0.17693797 -0.05913546  1.57664452] 9.131086926166788
3200 [ 0.47500188 -0.18169564  0.07772716  1.58469274] 8.034100980239648
3600 [ 0.47210938 -0.18683768 -0.01426454  1.59227341] 9.210340366976185
4000 [ 0.46944265 -0.19201379 -0.03007661  1.60005914] 9.21034031146506


array([ 0.46944265, -0.19201379, -0.03007661,  1.60005914])

In [17]:
eval_model_l2(X, y, 4000)

400 [ 0.4943937  -0.14214945  0.04006043  1.52574447] 4.06707498637855
800 [ 0.49264927 -0.14436713  0.03876325  1.52949262] 4.86122442514052
1200 [ 0.49090769 -0.14657897  0.04006813  1.53324553] 4.061826542867264
1600 [ 0.48916938 -0.14879119  0.04110281  1.53699889] 3.143440649436929
2000 [ 0.48741707 -0.15101601  0.0358113   1.54073803] 1.341956514199683
2400 [ 0.48568302 -0.15322035  0.04092003  1.54449906] 2.3370024521451316
2800 [ 0.48393576 -0.15544663  0.03467827  1.54823799] 1.253843256519901
3200 [ 0.48219078 -0.15766288  0.03335187  1.55198526] 1.167731308900485
3600 [ 0.48045047 -0.15987551  0.03365311  1.55573634] 1.1862443919625079
4000 [ 0.47872123 -0.16207413  0.04075069  1.55950128] 3.5386487557468262


array([ 0.47872123, -0.16207413,  0.04075069,  1.55950128])